In [1]:
import numpy as np

In [2]:
## Constants

service_times = {
    "contract"  :   30,
    "complain"  :   25,
    "confirm"   :   10,
    "request"   :   5, 
    "review"    :   10,
}

transition_matrices = {
    "A" : {
        "services": ["complain", "contract"], 
        "matrix" : [
            [0.9, 0.1], [0.2, 0.8],
        ],
        "time_between": 5
    },
    "B" : {
        "services": ["review", "request"], 
        "matrix" : [
            [0.95, 0.05], [0.15, 0.85],
        ],
        "time_between": 7
    },
    "C" : {
        "services": ["review", "request", "confirm"], 
        "matrix" : [
            [0.75, 0.15, 0.1], [0.1, 0.8, 0.1], [0.05, 0.05, 0.9],
        ],
        "time_between": 10
    },
}

interarrival_times = {
    "contract"  :   ["minute", "normal", 40, 36],
    "complain"  :   ["hour", "exponential", 0.5],
    "confirm"   :   ["hour", "gamma", 1, 2],
    "request"   :   ["minute", "exponential", 0.06], 
    "review"    :   ["minute", "normal", 15, 36],
}

queue_policies = {
    "contract"  :   'spt',
    "complain"  :   'fifo',
    "confirm"   :   'fifo',
    "request"   :   'spt',
    "review"    :   'siro',
}



In [3]:
class ServiceType:
    def __init__(self, name):
        self.name = name
        self.service_time = service_times[name]
        self.queue_policy = queue_policies[name]

    def get_service_time(self):
        scale = self.service_time
        sample = np.math.ceil(np.random.exponential(scale=scale))
        # print(scale, sample)
        return sample 
    
    def get_interarrival_time(self): 
        config = interarrival_times[self.name]
        scale, distr, params = config[0], config[1], config[2:]
        sample = 0
        if distr == "normal": 
            sample = np.random.normal(loc=params[0], scale=params[1]**0.5)
        elif distr == "exponential": 
            sample = np.random.exponential(scale=1/params[0])
        elif distr == "gamma": 
            sample = np.random.gamma(scale=params[0], shape=params[1])
        
        if scale == "hour": 
            sample *= 60
        elif scale == "second": 
            sample /= 60 
        
        return np.math.ceil(sample)        

class WorkerType: 
    def __init__(self, name):
        self.name = name
        config = transition_matrices[name]
        self.trans_dict = {}
        services = config["services"]
        for i in range(len(services)) :
            self.trans_dict[services[i]] = config["matrix"][i]
        self.transition_time = config["time_between"]
    
    def get_first_job(self): 
        # print(self.trans_dict.keys().)
        job_name = np.random.choice(list(self.trans_dict.keys()))
        return job_name
    
    def get_next_job(self, job): 
        next_job = np.random.choice(list(self.trans_dict.keys()), p=self.trans_dict[job])
        if next_job == job: 
            return None 
        return next_job
          
class Worker: 
    def __init__(self, type: WorkerType):
        self.type = type
        self.current_job = type.get_first_job()
        self.current_customer = None 
        self.time_to_move = type.transition_time
    
    def customer_finished(self): 
        if self.current_customer: 
            self.current_customer.proceed_work()
            if self.current_customer.is_finished(): 
                self.current_customer = None 

    def maybe_move(self): 
        self.time_to_move -= 1
        if self.time_to_move == 0: 
            self.time_to_move = self.type.transition_time
            next_job = self.type.get_next_job(self.current_job)
            if next_job: 
                customer = self.current_customer
                self.current_customer = None 
                self.current_job = next_job
                return customer
    
    def needs_customer(self): 
        if self.current_customer != None: 
            return False 
        return self.current_job
    
    def take_customer(self, customer): 
        self.current_customer = customer
        # print(str(self))
            
    def __str__(self): 
        return f"{self.type.name}:{self.current_job} => {self.current_customer}"



class Customer: 
    def __init__(self, type: ServiceType, enter_time): 
        self.type = type 
        self.enter_time = enter_time
        self.service_time = type.get_service_time()
        print(f"new_customer: {type.name}, {self.service_time}")
        self.remainig_time = self.service_time

    def is_finished(self): 
        return self.remainig_time == 0
    
    def proceed_work(self): 
        self.remainig_time -= 1

    def __str__(self): 
        return f"{self.type.name}"



class Service: 
    def __init__(self, type: ServiceType):
        self.type = type 
        self.queue = []
        self.customers = []
        self.next_customer = self.type.get_interarrival_time()
        self.A = [self.next_customer]

    def customer_arrived(self, i): 
        self.next_customer -= 1
        if self.next_customer == 0: 
            self.next_customer = self.type.get_interarrival_time()
            self.A.append(self.next_customer)
            customer = Customer(self.type, i)
            self.customers.append(customer)
            self.enqueue(customer)
    
    def enqueue(self, customer): 
        self.queue.append(customer)

    def dequeue(self): 
        if len(self.queue) == 0: 
            return None 
        policy = self.type.queue_policy
        if policy == 'fifo': 
            return self.queue.pop(0)
        elif policy == 'stp': 
            self.queue.sort(key=lambda x: x.remainig_time)
            return self.queue.pop(0)
        elif policy == 'siro': 

            return self.queue.pop(np.random.randint(0, len(self.queue)))
        
    def get_service_times(self): 
        s = [c.service_time - c.remainig_time for c in self.customers]
        return s




        

In [7]:
def simulate(workers_count, simulation_time):

    services = {}
    workers = []

    for name in service_times: 
        service_type = ServiceType(name)
        service = Service(service_type)
        services[name] = service

    for job in transition_matrices: 
        worker_type = WorkerType(job)
        for i in range(workers_count): 
            w = Worker(worker_type)
            workers.append(w)

    for i in range(simulation_time): # each iteratnio is 1 minute
        # print(f"============= {i} ===============")
        # print([str(worker) for worker in workers])
        
        # add new customers 
        for service in services.values(): 
            service.customer_arrived(i)

        # remove finished customers
        for worker in workers: 
            worker.customer_finished() 

        # move workers
        for worker in workers: 
            customer = worker.maybe_move()
            if customer: 
                services[customer.type.name].enqueue(customer)

        # update queues
        for worker in workers: 
            job = worker.needs_customer()
            if job: 
                customer = services[job].dequeue()
                # print(customer)
                if customer: 
                    worker.take_customer(customer)

    return services, workers
                






In [8]:
services, workers = simulate(2, 200)
A = {}
S = {}
for service in services: 
    A[service] = services[service].A
    
    S[service] = services[service].get_service_times()


print(A)
print(S)

new_customer: request, 4
new_customer: review, 8
new_customer: review, 4
new_customer: contract, 26
new_customer: review, 6
new_customer: request, 1
new_customer: request, 1
new_customer: review, 6
new_customer: request, 8
new_customer: contract, 48
new_customer: request, 11
new_customer: review, 4
new_customer: review, 7
new_customer: review, 9
new_customer: review, 11
new_customer: contract, 5
new_customer: request, 5
new_customer: review, 14
new_customer: request, 5
new_customer: complain, 67
new_customer: review, 20
new_customer: request, 1
new_customer: contract, 105
new_customer: review, 7
new_customer: review, 7
new_customer: contract, 24
{'contract': [33, 41, 44, 42, 34, 38], 'complain': [150, 352], 'confirm': [201], 'request': [1, 47, 3, 9, 19, 42, 15, 23, 49], 'review': [11, 16, 10, 17, 27, 14, 12, 2, 21, 25, 14, 22, 18]}
{'contract': [0, 0, 0, 0, 0], 'complain': [50], 'confirm': [], 'request': [0, 0, 0, 0, 0, 0, 0, 0], 'review': [8, 4, 6, 6, 4, 7, 9, 11, 14, 20, 7, 7]}
